In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from copulas.multivariate import GaussianMultivariate

# ==== 共通データ生成 ====
np.random.seed(42)
mean = np.zeros(5)
cov = np.array([
    [0.04, 0.02, 0.01, 0.015, 0.005],
    [0.02, 0.05, 0.02, 0.01, 0.005],
    [0.01, 0.02, 0.03, 0.01, 0.004],
    [0.015, 0.01, 0.01, 0.06, 0.007],
    [0.005, 0.005, 0.004, 0.007, 0.02]
])
returns = pd.DataFrame(np.random.multivariate_normal(mean, cov, 100),
                       columns=[f'Asset_{i+1}' for i in range(5)])
weights = np.array([0.2] * 5)
n_sim = 1000

# ==== 各種手法定義 ====

def bootstrap_multivariate(returns, weights, n_sim=1000):
    sim_returns = []
    for _ in range(n_sim):
        sample = returns.sample(n=len(returns), replace=True)
        port_ret = sample @ weights
        cum_ret = (1 + port_ret).prod() - 1
        sim_returns.append(cum_ret)
    return np.array(sim_returns)

def moving_block_bootstrap(returns, weights, block_size=5, n_sim=1000):
    n_obs = len(returns)
    n_blocks = n_obs // block_size
    sim_returns = []
    for _ in range(n_sim):
        resampled = []
        for _ in range(n_blocks):
            start = np.random.randint(0, n_obs - block_size + 1)
            block = returns.iloc[start:start+block_size]
            resampled.append(block)
        sampled = pd.concat(resampled, ignore_index=True)
        port_ret = sampled @ weights
        cum_ret = (1 + port_ret).prod() - 1
        sim_returns.append(cum_ret)
    return np.array(sim_returns)

def stationary_bootstrap(returns, weights, p=0.1, n_sim=1000):
    n_obs = len(returns)
    sim_returns = []
    for _ in range(n_sim):
        i = np.random.randint(n_obs)
        series = []
        while len(series) < n_obs:
            block_len = np.random.geometric(p)
            end = min(i + block_len, n_obs)
            series.append(returns.iloc[i:end])
            i = np.random.randint(n_obs)
        sampled = pd.concat(series, ignore_index=True).iloc[:n_obs]
        port_ret = sampled @ weights
        cum_ret = (1 + port_ret).prod() - 1
        sim_returns.append(cum_ret)
    return np.array(sim_returns)

def empirical_transform(df):
    return df.rank(method='average') / (len(df) + 1)

def copula_bootstrap(returns, weights, n_sim=1000):
    u_data = empirical_transform(returns)
    copula = GaussianMultivariate()
    copula.fit(u_data)
    u_samples = copula.sample(n_sim)
    simulated_returns = pd.DataFrame(index=range(n_sim), columns=returns.columns)
    for col in returns.columns:
        u_vals = u_samples[col].values
        ecdf = np.sort(returns[col].values)
        simulated_returns[col] = np.quantile(ecdf, u_vals)
    port_ret = simulated_returns @ weights
    return port_ret.values

def parametric_monte_carlo(returns, weights, n_sim=1000):
    mean_vec = returns.mean().values
    cov_mat = returns.cov().values
    sims = np.random.multivariate_normal(mean_vec, cov_mat, size=n_sim)
    port_ret = sims @ weights
    return port_ret

# ==== 指標計算 ====

def compute_stats(sim_returns):
    return {
        'mean': np.mean(sim_returns),
        'std': np.std(sim_returns),
        'VaR_5': np.percentile(sim_returns, 5),
        'CVaR_5': np.mean(sim_returns[sim_returns <= np.percentile(sim_returns, 5)]),
        'skew': skew(sim_returns),
        'kurtosis': kurtosis(sim_returns)
    }

# ==== 実行と比較 ====

results = {
    'Multivariate': bootstrap_multivariate(returns, weights, n_sim),
    'Moving Block': moving_block_bootstrap(returns, weights, block_size=5, n_sim=n_sim),
    'Stationary': stationary_bootstrap(returns, weights, p=0.1, n_sim=n_sim),
    'Copula': copula_bootstrap(returns, weights, n_sim=n_sim),
    'Parametric': parametric_monte_carlo(returns, weights, n_sim=n_sim),
}

stats_df = pd.DataFrame({name: compute_stats(ret) for name, ret in results.items()}).T

import seaborn as sns
import matplotlib.pyplot as plt
import ace_tools as tools

# ヒストグラム描画
plt.figure(figsize=(12, 6))
for name, ret in results.items():
    sns.kdeplot(ret, label=name)
plt.title("Simulated Portfolio Return Distributions")
plt.xlabel("Cumulative Return")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 統計指標出力
tools.display_dataframe_to_user(name="Bootstrapped Portfolio Return Statistics", dataframe=stats_df.round(4))
